In [1]:
import pandas as pd
import sqlite3 
import pytz
import datetime
tz = pytz.timezone('UTC')
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_colwidth", 200)

In [2]:
def cnts_pcts(series):
    cnts = series.value_counts()
    pcts = 100*series.value_counts(normalize=True).round(5)
    combo = pd.concat({"Count":cnts, "Percent": pcts}, axis=1)
    return(combo)


In [3]:
con = sqlite3.connect('/home/malcolm/petfinder/data/petfinder.db')
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('dog_10475_30mi',), ('cat_10475_30mi',)]


In [4]:
cursor.execute("""
select min(date_saved), max(date_saved), count(*)
from dog_10475_30mi
""").fetchall()

[('2021-07-03', '2021-09-04', 149204)]

In [5]:
cursor.execute("""
select date_saved, count(*) as cnt_dogs, count(distinct id) as cnt_uniqueids
from dog_10475_30mi
group by date_saved
order by 1 desc
""").fetchall()

[('2021-09-04', 2382, 2378),
 ('2021-09-03', 2345, 2345),
 ('2021-09-02', 2271, 2271),
 ('2021-09-01', 2260, 2259),
 ('2021-08-31', 2217, 2216),
 ('2021-08-30', 2228, 2224),
 ('2021-08-29', 2274, 2273),
 ('2021-08-28', 2292, 2290),
 ('2021-08-27', 2293, 2293),
 ('2021-08-26', 2293, 2292),
 ('2021-08-25', 2288, 2287),
 ('2021-08-24', 2297, 2296),
 ('2021-08-23', 2279, 2275),
 ('2021-08-22', 2265, 2263),
 ('2021-08-21', 2309, 2306),
 ('2021-08-20', 2281, 2278),
 ('2021-08-19', 2275, 2275),
 ('2021-08-18', 2413, 2405),
 ('2021-08-17', 2298, 2298),
 ('2021-08-16', 2290, 2290),
 ('2021-08-15', 2320, 2317),
 ('2021-08-14', 2367, 2366),
 ('2021-08-13', 2353, 2350),
 ('2021-08-12', 2339, 2338),
 ('2021-08-11', 2339, 2338),
 ('2021-08-10', 2340, 2339),
 ('2021-08-09', 2368, 2366),
 ('2021-08-08', 2359, 2354),
 ('2021-08-07', 2428, 2428),
 ('2021-08-06', 2423, 2421),
 ('2021-08-05', 2399, 2398),
 ('2021-08-04', 2404, 2404),
 ('2021-08-03', 2392, 2391),
 ('2021-08-02', 2400, 2399),
 ('2021-08-01'

In [6]:
today = datetime.datetime.now()
today_str = str(datetime.datetime.now().date())
two_weeks = today - datetime.timedelta(days=7)
two_weeks_str = str(two_weeks.date())

In [7]:
data_raw = pd.read_sql(f"""
select * from dog_10475_30mi
where date_saved between '{two_weeks_str}' and '{today_str}'
""" , con)
data_raw['published_at'] = pd.to_datetime(data_raw['published_at'])
data_raw['status_changed_at'] = pd.to_datetime(data_raw['status_changed_at'])
data_raw['date_saved'] = pd.to_datetime(data_raw['date_saved'], utc=True)

data = data_raw[data_raw['date_saved'] == data_raw['date_saved'].max()]
print(data.shape)
data.head()

(2382, 50)


,id,organization_id,url,type,species,age,gender,size,coat,tags,name,description,organization_animal_id,photos,videos,status,status_changed_at,published_at,distance,breeds.primary,breeds.secondary,breeds.mixed,breeds.unknown,colors.primary,colors.secondary,colors.tertiary,attributes.spayed_neutered,attributes.house_trained,attributes.declawed,attributes.special_needs,attributes.shots_current,environment.children,environment.dogs,environment.cats,primary_photo_cropped.small,primary_photo_cropped.medium,primary_photo_cropped.large,primary_photo_cropped.full,contact.email,contact.phone,contact.address.address1,contact.address.address2,contact.address.city,contact.address.state,contact.address.postcode,contact.address.country,animal_id,animal_type,primary_photo_cropped,date_saved
15887,52875185,NY920,https://www.petfinder.com/dog/peri-52875185/ny/staten-island/louies-legacy-animal-rescue-ny920/?referrer_id=a0b0e7e5-a0ff-4872-86f9-c0328ab65a7c,Dog,Dog,Baby,Female,Medium,None,"0 []\n1 [Friendly, Affectionate, Loyal, Gentle, Playfu...\n2 [Friendly, Affectionate, Loyal, Gentle, Playfu...\n3 [Friendly, ...",Peri,None,LOUE-A-11860,0 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n1 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n2 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n3 [{'small': ...,0 []\n1 []\n2 []\n3 ...,adoptable,2021-09-04 06:20:28+00:00,2021-09-04 06:20:28+00:00,23.3185,Australian Kelpie,None,1,0,Brown / Chocolate,White / Cream,None,1,0,None,0,1,NaN,NaN,NaN,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/52875185/1/?bust=1630737038&width=300,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/52875185/1/?bust=1630737038&width=450,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/52875185/1/?bust=1630737038&width=600,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/52875185/1/?bust=1630737038,newyork@louieslegacy.org,(646) 397-5527,None,None,Staten Island,NY,10302,US,52875185,dog,None,2021-09-04 00:00:00+00:00
15888,52873226,NY1425,https://www.petfinder.com/dog/karl-52873226/ny/oakland-gardens/peytons-safe-haven-inc-ny1425/?referrer_id=a0b0e7e5-a0ff-4872-86f9-c0328ab65a7c,Dog,Dog,Baby,Male,Medium,Short,"0 []\n1 [Friendly, Affectionate, Loyal, Gentle, Playfu...\n2 [Friendly, Affectionate, Loyal, Gentle, Playfu...\n3 [Friendly, ...",Karl,If big ears wasn&#039;t cute enough put them on a lowrider and you get these gorgeous boys !!! They are...,None,0 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n1 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n2 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n3 [{'small': ...,0 []\n1 []\n2 []\n3 ...,adoptable,2021-09-04 02:19:51+00:00,2021-09-04 02:19:51+00:00,9.8217,Cattle Dog,None,1,0,Black,None,None,1,0,None,0,1,1.0,1.0,1.0,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/52873226/1/?bust=1630721989&width=300,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/52873226/1/?bust=1630721989&width=450,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/52873226/1/?bust=1630721989&width=600,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/52873226/1/?bust=1630721989,info@peytonssafehaven.org,None,None,None,Oakland Gardens,NY,11364,US,52873226,dog,None,2021-09-04 00:00:00+00:00
15889,52873222,NY1425,https://www.petfinder.com/dog/kevin-52873222/ny/oakland-gardens/peytons-safe-haven-inc-ny1425/?referrer_id=a0b0e7e5-a0ff-4872-86f9-c0328ab65a7c,Dog,Dog,Baby,Male,Medium,Short,"0 []\n1 [Friendly, Affectionate, Loyal, Gentle, Playfu...\n2 [Friendly, Affectionate, Loyal, Gentle, Playfu...\n3 [Friendly, ...",Kevin,If big ears wasn&#039;t cute enough put them on a lowrider and you get these gorgeous boys !!! They are...,None,0 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n1 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n2 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n3 [{'small': ...,0 []\n1 []\n2 []\n3 ...,adoptable,2021-09-04 02:18:33+00:00,2021-09-04 02:18:33+00:00,9.8217,Cattle Dog,None,1,0,Apricot / Beige,White / Cream,None,1,0,None,0,1,1.0,1.0,1.0,https://dl5zpyw5k3jeb.clo

## Peak at some attributes

In [8]:
data.columns

Index(['id', 'organization_id', 'url', 'type', 'species', 'age', 'gender',
       'size', 'coat', 'tags', 'name', 'description', 'organization_animal_id',
       'photos', 'videos', 'status', 'status_changed_at', 'published_at',
       'distance', 'breeds.primary', 'breeds.secondary', 'breeds.mixed',
       'breeds.unknown', 'colors.primary', 'colors.secondary',
       'colors.tertiary', 'attributes.spayed_neutered',
       'attributes.house_trained', 'attributes.declawed',
       'attributes.special_needs', 'attributes.shots_current',
       'environment.children', 'environment.dogs', 'environment.cats',
       'primary_photo_cropped.small', 'primary_photo_cropped.medium',
       'primary_photo_cropped.large', 'primary_photo_cropped.full',
       'contact.email', 'contact.phone', 'contact.address.address1',
       'contact.address.address2', 'contact.address.city',
       'contact.address.state', 'contact.address.postcode',
       'contact.address.country', 'animal_id', 'animal_type',

In [9]:
data['status'].value_counts()

adoptable    2382
Name: status, dtype: int64

In [10]:
data['status_changed_at'] 
data['published_at'].min()

Timestamp('2010-01-17 00:00:00+0000', tz='UTC')

In [11]:
data[data['status_changed_at'] <= '2021-01-01'].shape
# Probably data quality issue.. should investigate or drop these records

(309, 50)

In [12]:
status_change = data[data['status_changed_at'] != data['published_at']]
status_change.shape

(465, 50)

In [13]:
status_change

,id,organization_id,url,type,species,age,gender,size,coat,tags,name,description,organization_animal_id,photos,videos,status,status_changed_at,published_at,distance,breeds.primary,breeds.secondary,breeds.mixed,breeds.unknown,colors.primary,colors.secondary,colors.tertiary,attributes.spayed_neutered,attributes.house_trained,attributes.declawed,attributes.special_needs,attributes.shots_current,environment.children,environment.dogs,environment.cats,primary_photo_cropped.small,primary_photo_cropped.medium,primary_photo_cropped.large,primary_photo_cropped.full,contact.email,contact.phone,contact.address.address1,contact.address.address2,contact.address.city,contact.address.state,contact.address.postcode,contact.address.country,animal_id,animal_type,primary_photo_cropped,date_saved
16216,52821522,NY187,https://www.petfinder.com/dog/lucylou-52821522/ny/harrison/pet-rescue-ny187/?referrer_id=a0b0e7e5-a0ff-4872-86f9-c0328ab65a7c,Dog,Dog,Adult,Female,Small,Short,"0 []\n1 [Friendly, Affectionate, Loyal, Gentle, Playfu...\n2 [Friendly, Affectionate, Loyal, Gentle, Playfu...\n3 [Friendly, ...",LucyLou,"We all love you, LucyLou. She is a personable, easy going, friendly girl. LucyLou enjoys her buddy walks at petrescue...",None,0 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n1 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n2 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n3 [{'small': ...,0 []\n1 []\n2 []\n3 ...,adoptable,2021-09-02 23:17:35+00:00,2021-08-30 19:19:48+00:00,8.3540,Terrier,Chihuahua,1,0,Apricot / Beige,None,None,1,0,None,0,1,NaN,1.0,NaN,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/52821522/1/?bust=1630351185&width=300,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/52821522/1/?bust=1630351185&width=450,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/52821522/1/?bust=1630351185&width=600,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/52821522/1/?bust=1630351185,nypetrescue@gmail.com,(914) 835-3332,7 Harrison Avenue,None,Harrison,NY,10528,US,52821522,dog,None,2021-09-04 00:00:00+00:00
16305,52797518,NY486,https://www.petfinder.com/dog/kali-52797518/ny/east-rockaway/fur-babies-ny486/?referrer_id=a0b0e7e5-a0ff-4872-86f9-c0328ab65a7c,Dog,Dog,Young,Female,Small,Short,"0 []\n1 [Friendly, Affectionate, Loyal, Gentle, Playfu...\n2 [Friendly, Affectionate, Loyal, Gentle, Playfu...\n3 [Friendly, ...",Kali,Kali is a 1 yr old 25 lb female Boston staffy mix. She is good with other dogs and kids....,None,0 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n1 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n2 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n3 [{'small': ...,0 []\n1 []\n2 []\n3 ...,adoptable,2021-09-04 03:51:32+00:00,2021-08-27 21:22:35+00:00,18.2310,Boston Terrier,Staffordshire Bull Terrier,1,0,Black,White / Cream,None,1,1,None,0,1,1.0,1.0,NaN,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/52797518/1/?bust=1630099351&width=300,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/52797518/1/?bust=1630099351&width=450,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/52797518/1/?bust=1630099351&width=600,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/52797518/1/?bust=1630099351,Kobemnstr@hotmail.com,None,None,None,East Rockaway,NY,11518,US,52797518,dog,None,2021-09-04 00:00:00+00:00
16328,52794188,NY1270,https://www.petfinder.com/dog/dillon-52794188/ny/new-york/pupstarz-rescue-ny1270/?referrer_id=a0b0e7e5-a0ff-4872-86f9-c0328ab65a7c,Dog,Dog,Baby,Male,Medium,None,"0 []\n1 [Friendly, Affectionate, Loyal, Gentle, Playfu...\n2 [Friendly, Affectionate, Loyal, Gentle, Playfu...\n3 [Friendly, ...",Dillon,"Meet Dillon, a 14 week old, 15 pound (as of 9/3), adorable Shepherd/heeler mix. This delightful puppy is soft, sweet,...",None,0 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n1 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n2 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n3 [{'small': ...,0 []\n1 []\n2 []\n3 ...,adoptable,2021-09-03 16:38:10+00:00,2021-08-27 17:04:45+00:00,12.2658,Shepherd,Australi

In [14]:
status_change.status.value_counts()

adoptable    465
Name: status, dtype: int64

In [15]:
data['colors.secondary'].value_counts()

White / Cream                       359
Yellow / Tan / Blond / Fawn          97
Black                                75
Brown / Chocolate                    62
Red / Chestnut / Orange              21
Gray / Blue / Silver                 14
Brindle                              14
Bicolor                               8
Tricolor (Brown, Black, & White)      8
Apricot / Beige                       7
Sable                                 4
Golden                                3
Merle (Blue)                          2
Merle (Red)                           1
Name: colors.secondary, dtype: int64

In [16]:
data['age'].value_counts()

Adult     1011
Young      587
Baby       572
Senior     212
Name: age, dtype: int64

In [17]:
data['organization_id'].value_counts()

NY1274    232
NY949     136
NY936     106
NY1303     70
NY359      55
         ... 
NY1487      1
NJ08        1
NY587       1
NY1110      1
NY24        1
Name: organization_id, Length: 180, dtype: int64

In [18]:
data.iloc[1]

id                                                                                                                                                                                                                             52873226
organization_id                                                                                                                                                                                                                  NY1425
url                                                                                      https://www.petfinder.com/dog/karl-52873226/ny/oakland-gardens/peytons-safe-haven-inc-ny1425/?referrer_id=a0b0e7e5-a0ff-4872-86f9-c0328ab65a7c
type                                                                                                                                                                                                                                Dog
species                                                                 

## Dog Status Change

In [19]:
def dog_days_gb_func(df, adopt_date=(datetime.datetime.now() - datetime.timedelta(days=1)).date()):
    
    df = df.sort_values('date_saved')
    
    out = {}
    out['Days in DB (yourdb)'] = (df['date_saved'].max() - df['date_saved'].min()).days
    out['Count'] = df.shape[0]
    out['Days ad posted'] = (df['date_saved'].max() - df['published_at'].min()).days
    adoptable_days = df[df['status'] == 'adoptable']
    out['Days adoptable'] = adoptable_days.shape[0]
    out['Last Adoptable Day'] = adoptable_days['date_saved'].max()
    
    out['Adopted (last status)'] = (df.iloc[-1].status != 'adoptable')
    out['Adopted (last date)'] = df.iloc[-1]['date_saved'] >= adopt_date 
    
    out_series = pd.Series(out)
    last_record = df.iloc[0]
    out_all = pd.concat([out_series, last_record]) 
    return(out_all)
    

In [20]:
dog_gb = data_raw.groupby('id').apply(dog_days_gb_func)
print("Shape: ", dog_gb.shape)
dog_gb

Shape:  (2764, 57)


,Days in DB (yourdb),Count,Days ad posted,Days adoptable,Last Adoptable Day,Adopted (last status),Adopted (last date),id,organization_id,url,type,species,age,gender,size,coat,tags,name,description,organization_animal_id,photos,videos,status,status_changed_at,published_at,distance,breeds.primary,breeds.secondary,breeds.mixed,breeds.unknown,colors.primary,colors.secondary,colors.tertiary,attributes.spayed_neutered,attributes.house_trained,attributes.declawed,attributes.special_needs,attributes.shots_current,environment.children,environment.dogs,environment.cats,primary_photo_cropped.small,primary_photo_cropped.medium,primary_photo_cropped.large,primary_photo_cropped.full,contact.email,contact.phone,contact.address.address1,contact.address.address2,contact.address.city,contact.address.state,contact.address.postcode,contact.address.country,animal_id,animal_type,primary_photo_cropped,date_saved
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
15546918,7,8,4248,8,2021-09-04 00:00:00+00:00,False,True,15546918,NY769,https://www.petfinder.com/dog/buddy-15546918/ny/glen-head/shepherds-hope-rescue-ny769/?referrer_id=a0b0e7e5-a0ff-4872-86f9-c0328ab65a7c,Dog,Dog,Young,Male,Medium,None,"0 [Friendly, Affectionate, Loyal, Gentle, Playfu...\n1 [Friendly, Affectionate, Loyal, Gentle, Playfu...\n2 [Friendly, Affectionate, Playful, Smart, Funny...\n3 [Friendly, ...",Buddy,"Here&#039;s Buddy, an adorable 8 year old Shepherd mix. He was adopted from one of the city shelters to an...",825418,0 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n1 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n2 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n3 [{'small': ...,"0 []\n1 [{'embed': '<iframe title=""Video"" frameborder=...\n2 []\n3 ...",adoptable,2010-01-17 20:48:17+00:00,2010-01-17 00:00:00+00:00,10.9750,Shepherd,None,1,0,None,None,None,1,1,None,0,1,0.0,0.0,0.0,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/15546918/3/?bust=1452873122&width=300,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/15546918/3/?bust=1452873122&width=450,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/15546918/3/?bust=1452873122&width=600,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/15546918/3/?bust=1452873122,toomanyshepherds@aol.com,None,None,None,Glen Head,NY,11545,US,15546918,dog,None,2021-08-28 00:00:00+00:00
25009612,7,8,3167,8,2021-09-04 00:00:00+00:00,False,True,25009612,NY955,https://www.petfinder.com/dog/franklin-25009612/ny/glen-cove/project-pet-inc-ny955/?referrer_id=a0b0e7e5-a0ff-4872-86f9-c0328ab65a7c,Dog,Dog,Senior,Male,Medium,Short,"0 [Friendly, Affectionate, Loyal, Gentle, Playfu...\n1 [Friendly, Affectionate, Loyal, Gentle, Playfu...\n2 [Friendly, Affectionate, Playful, Smart, Funny...\n3 [Friendly, ...",Franklin,PLEASE EMAIL: mbwambwa@icloud.com\nDIRECTLY TO FIND OUT HOW TO MEET THIS CUTIE!\n\nThis 8 year-old love muffin is summed up...,None,0 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n1 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n2 [{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...\n3 [{'small': ...,"0 []\n1 [{'embed': '<iframe title=""Video"" frameborder=...\n2 []\n3 ...",adoptable,2014-09-29 20:37:01+00:00,2013-01-01 16:05:32+00:00,10.6483,Terrier,None,1,0,Yellow / Tan / Blond / Fawn,White / Cream,None,1,1,None,0,1,NaN,1.0,NaN,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/25009612/4/?bust=1533002011&width=300,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/25009612/4/?bust=1533002011&width=450,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/25009612/4/?bust=1533002011&width=600,https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/25009612/4/?bust=1533002011,friends@projectpetinc.org,None,None,None,Glen Cove,NY,11542,US,25009612,dog,None,2021-08-28 00:00:00+00:00
25777517,1,2,3074,2,2021-09-04 00:00:00+00:00,False,True,25777517,NY977,https://www.petfinder.com/dog/italian-greyhound-place-25777517/ny/croton-on-hudson/italian-greyhound-place-ny977/?referrer_id=a0b0e7e5-a0ff-4872-86f9-c0328ab65a7c,Dog,Dog,Adult,Male,Small,

In [21]:
dog_gb['Days in DB (yourdb)'].describe()

count    2764.000000
mean        5.612156
std         2.392701
min         0.000000
25%         5.000000
50%         7.000000
75%         7.000000
max         7.000000
Name: Days in DB (yourdb), dtype: float64

In [22]:
dog_gb['Days ad posted'].describe()

count    2764.000000
mean      159.111433
std       360.458372
min        -1.000000
25%        10.000000
50%        41.000000
75%       122.000000
max      4248.000000
Name: Days ad posted, dtype: float64

In [23]:
long_ad_posted_dogs = dog_gb[dog_gb['Days ad posted'] >= 180]
long_ad_posted_dogs.shape

(489, 57)

In [24]:
long_ad_posted_dogs['age'].value_counts()

Adult     331
Young     101
Senior     48
Baby        9
Name: age, dtype: int64

In [25]:
age_cnts = cnts_pcts(dog_gb['age'])
age_cnts

,Count,Percent
Adult,1112,40.232
Baby,739,26.737
Young,687,24.855
Senior,226,8.177


In [26]:
long_ad_posted_age_cnts = cnts_pcts(long_ad_posted_dogs['age'])
long_ad_posted_age_cnts['Sample Ratio'] = long_ad_posted_age_cnts['Percent']/age_cnts['Percent']
long_ad_posted_age_cnts

,Count,Percent,Sample Ratio
Adult,331,67.689,1.682467
Young,101,20.654,0.830980
Senior,48,9.816,1.200440
Baby,9,1.840,0.068818
